# Confiabilidade de viga mista

## Importando bibliotecas

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

## Definindo funções

In [2]:
def calculo_destribuicao_media(media: float, desvpad: float, area: bool = False) -> list:
    amostra: int = 1000
    medias: list = []
    areas: list = []
    
    if (area):
        for i in range(amostra):
            numero_aleatorio: float = np.random.rand()
            medias.append(norm.ppf(numero_aleatorio, media, desvpad))
            areas.append(np.pi * (medias[i] / 2) ** 2)
        
        return medias, areas

    else:
        for i in range(amostra):
            numero_aleatorio: float = np.random.rand()
            medias.append(norm.ppf(numero_aleatorio, media, desvpad))
            
        return medias
        

## Médias - Espessura da laje

In [3]:
espessura_laje: float = 10
desvpad_espessura_laje: float = 0.06 * espessura_laje

espessuras_laje: list = calculo_destribuicao_media(espessura_laje, desvpad_espessura_laje)

## Médias - Limite de escoamento do perfil de aço

In [4]:
limite_escoamento_perfil: float = 250 * 1.08
desvpad_escoamento_perfil: float = 0.08 * limite_escoamento_perfil

limites_escoamento_perfil: list = calculo_destribuicao_media(limite_escoamento_perfil, desvpad_escoamento_perfil)

## Médias - Limite de escoamento da armadura

In [5]:
limite_escoamento_armadura: float = 500 * 1.08
desvpad_escoamento_armadura: float = 0.08 * limite_escoamento_armadura

limites_escoamento_perfil: list = calculo_destribuicao_media(limite_escoamento_armadura, desvpad_escoamento_armadura)

## Médias - Resistência do concreto à compressão

In [6]:
resistencia_concreto: float = 20 * 1.17
desvpad_resistencia_concreto: float = 0.15 * resistencia_concreto

resistencias_concreto: list = calculo_destribuicao_media(resistencia_concreto, desvpad_resistencia_concreto)

## Médias - Diamêtro do rebite

In [7]:
diametro_rebite: float = 1.59
desvpad_diametro_rebite: float = 0.01 * diametro_rebite

diametros_rebites, areas_rebites = calculo_destribuicao_media(diametro_rebite, desvpad_diametro_rebite, True)


## Médias - Resistência do rebite

In [8]:
resistencia_rebite: float = 1.59
desvpad_resistencia_rebite: float = 0.01 * resistencia_rebite

resistencias_rebite: list = calculo_destribuicao_media(resistencia_rebite, desvpad_resistencia_rebite)

## Pré-dimensionamento

In [17]:
carga_permanente_g1: float = 7.6
carga_permanente_g2: float = 5

variacao_carga: list = [5, 9]
carga_utilizacao_q2: list = [i for i in range(variacao_carga[0], variacao_carga[1])]

carga_construcao_q1: list = [i / 5.6 for i in carga_utilizacao_q2]

# Propriedades da laje
vao_laje: float = 9
espacamento_laje: float = 2.8

# Carga distrubuída do projeto
carga_distribuida_projeto: list = [1.4 * (carga_permanente_g1 + carga_permanente_g2) + carga_construcao_q1[i] * carga_utilizacao_q2[i] for i in range(len(carga_utilizacao_q2))]

# Momento solicitante do projeto
momento_solicitante_projeto: list = [(carga_distribuida_projeto[i] * vao_laje ** 2) / 8 for i in range(len(carga_distribuida_projeto))]

altura_laje_ht: float = 45
coeficiente_ya1: float = 1.1
x: float = 10

# Área mínima da seção de aço
areas_minimas: list = [[] for i in range(len(momento_solicitante_projeto))]
index_matriz: int = 0
for i in momento_solicitante_projeto:
    for j in range(len(limites_escoamento_perfil)):
        areas_minimas[index_matriz].append((i * 100) / (((limites_escoamento_perfil[j] / 10) / coeficiente_ya1) * ((altura_laje_ht / 2) + espessuras_laje[j] - (x / 2))))
    index_matriz += 1
    
print(areas_minimas)


[[], [], [], []]
[[18.084667296443214, 16.725421776037894, 17.466061118071323, 17.08721266863039, 14.33512320247834, 15.838377785888765, 16.110097750350672, 17.52404028478098, 16.30696297084121, 18.088924281293888, 16.57754588006089, 17.093713137694035, 15.066035737173747, 17.547155159301397, 14.532856876620135, 18.293791290882975, 16.521836309757177, 18.438827934373517, 15.605022016383671, 14.713444916441885, 15.359986826600894, 16.632177515246813, 18.841162573815694, 17.46704443524511, 15.946751684228799, 17.159807488413414, 17.55745524122463, 18.107321639677302, 14.882843844528319, 17.093020906932928, 17.25938673148142, 15.002552974122178, 14.537095674680819, 16.03003719463174, 19.393064479428066, 16.376907524946812, 18.593925836461377, 18.07569521220233, 16.457203141198395, 17.595249787367184, 16.606977675556575, 16.406993292240152, 16.78639077779923, 14.740450802416927, 15.413440723101193, 16.512014867360673, 19.17619046353192, 18.388596700737903, 14.641536101791537, 15.4439959746